# Import library

In [1]:
%load_ext autoreload
%autoreload 2

# speed up the loading of the training data
import cv2
import os
import numpy as np
import torch as th
import itertools
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from onlineTripletloss import *
from selector import *
from model_SAGAN1_1_2att import NetG, NetD, NetA
# from model_SAGAN2_Triplet import NetG, NetD, NetA
# from model_WGANGP import NetG, NetD, NetA
# from model_WGAN import NetG, NetD, NetA
# from model_siGAN import NetG, NetD, NetA
from dataset2Loader import CASIABDataset
# from dataset2Loader_newtriplet import CASIABDataset
# from dataset2Loader_triplet import CASIABDataset
import torch.optim as optim
import visdom
from torchvision.utils import make_grid
# Data_Dir = '../GaitRecognition/DatasetB_GEI_64x64_allseq/'
Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
Model_Name = 'Model_64x64_SAGAN_90_trial15'
Model_dir = './Transform_Model/'+ Model_Name
if not os.path.isdir(Model_dir):
    os.mkdir(Model_dir)

In [ ]:

Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
import os

for r, d, files in os.walk(Data_Dir):
    print(r)
    print(len(d))
    print(len(files))
    
# cpt = sum([len(files) for r, d, files in os.walk(Data_Dir)])
# cpt = sum([len(d) for r, d, files in os.walk(Data_Dir)])
# print(cpt)
# list = os.listdir(Data_Dir) # dir is your directory path
# number_files = len(list)
# print(number_files)

# import fnmatch
# print(len(fnmatch.filter(os.listdir(Data_Dir), '*.png')))

In [ ]:
dataset = CASIABDataset(data_dir=Data_Dir)
ass_label, noass_label, img = dataset.getbatch(16)

## iteration 刪

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
        init.xavier_normal_(tensor, gain=1.)
#         init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir)

iteration = 0
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 128

label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print('Training starts')
while iteration < 30000:
    ass_label, noass_label, img = dataset.getbatch(batchSize)
    ass_label = ass_label.to(device).to(th.float32)
    noass_label = noass_label.to(device).to(th.float32)
    img = img.to(device).to(th.float32)
    
    # update D
    lossD = 0
    optimD.zero_grad()
    output = netd(ass_label)
    label.fill_(real_label)
    lossD_real1 = F.binary_cross_entropy(output, label)
    lossD += lossD_real1.item()
    lossD_real1.backward()

    label.fill_(real_label)
    output1 = netd(noass_label)
    lossD_real2 = F.binary_cross_entropy(output1, label)
    lossD += lossD_real2.item()
    lossD_real2.backward()

    fake = netg(img).detach()
    label.fill_(fake_label)
    output2 = netd(fake)
    lossD_fake = F.binary_cross_entropy(output2, label)
    lossD += lossD_fake.item()
    lossD_fake.backward()

    optimD.step()
    
    # update A
    lossA = 0
    optimA.zero_grad()
    assd = th.cat((img, ass_label), 1)
    noassd = th.cat((img, noass_label), 1)
    fake = netg(img).detach()
    faked = th.cat((img, fake), 1)

    label.fill_(real_label)
    output1 = neta(assd)
    lossA_real1 = F.binary_cross_entropy(output1, label)
    lossA += lossA_real1.item()
    lossA_real1.backward()

    label.fill_(fake_label)
    output = neta(noassd)
    lossA_real2 = F.binary_cross_entropy(output, label)
    lossA += lossA_real2.item()
    lossA_real2.backward()

    label.fill_(fake_label)
    output = neta(faked)
    lossA_fake = F.binary_cross_entropy(output, label)
    lossA += lossA_fake.item()
    lossA_fake.backward()
    
    optimA.step()
    
    # update G
    lossG = 0
    optimG.zero_grad()
    fake = netg(img)
    output = netd(fake)

    label.fill_(real_label)
    lossGD = F.binary_cross_entropy(output, label)
    lossG += lossGD.item()
    lossGD.backward(retain_graph=True)

    faked = th.cat((img, fake), 1)
    output = neta(faked)
    label.fill_(real_label)
    lossGA = F.binary_cross_entropy(output, label)
    lossG += lossGA.item()
    lossGA.backward()
    
    optimG.step()

    iteration += 1
    
    if iteration % 20 == 0:
        with th.no_grad():
            netg.eval()  #切換
            fake = netg(img) 25
            netg.train() #切換回去
        fake = (fake + 1) / 2 * 255
        real = (ass_label + 1) / 2 * 255
        ori = (img + 1) / 2 * 255
        al = th.cat((fake, real, ori), 2)
        display = make_grid(al, 20).cpu().numpy()
        if win1 is None:
            win1 = vis.image(display,
                             opts=dict(title="train", caption='train'))
        else:
            vis.image(display, win=win1)
    if iteration % 1000 == 0 or iteration==500:
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_Name+'/snapshot'+ Model_Name +'_%d.t7' % iteration)

    if iteration % 100==0 or iteration==10 :
        if win is None:
            win = vis.line(X=np.array([[iteration, iteration,
                                        iteration]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='iterations',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[iteration, iteration,
                                  iteration]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')
            
        with open(Model_Name+"/snapshot_log.txt", "a") as myfile:
            myfile.write('iter = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            iteration, lossG/2, lossA/3, lossD/3
        ))
        print('iter = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            iteration, lossG/2, lossA/3, lossD/3
        ))


## Epoch

In [ ]:
dataset = CASIABDataset(data_dir=Data_Dir)
train_loader = th.utils.data.DataLoader(dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=False)
train_loader.next()

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 0
n_g = 0
n_d = 0


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=glr/2)
optimD = optim.Adam(netd.parameters(), lr=dlr/3)
optimA = optim.Adam(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

print('Training starts')
low_loss = 10
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#     for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
#         ass_img = ass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
#         noass_img = noass_img.to(device).to(th.float32)

        if i % n_g ==0:
            # update D
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake = netg(img).detach()
            label.fill_(fake_label)
            output2 = netd(fake)
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            fake = netg(img).detach()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()

        if i % n_d ==0: 
            # update G
            lossG = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

            label.fill_(real_label)
            lossGD = F.binary_cross_entropy(output, label)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
            label.fill_(real_label)
            lossGA = F.binary_cross_entropy(output, label)
            lossG += lossGA.item()
            lossGA.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:  
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
    
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        



## update k times

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 0
n_g = 2
n_d = 0


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=glr/2)
optimD = optim.Adam(netd.parameters(), lr=dlr/3)
optimA = optim.Adam(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10
print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#     for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
#         noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
#         ass_img = ass_img.to(device).to(th.float32)
        
        if i % n_g ==0:
            # update D
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake = netg(img).detach()
            label.fill_(fake_label)
            output2 = netd(fake)
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
        if i % n_d ==0: 
            # update G
            lossG = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

            label.fill_(real_label)
            lossGD = F.binary_cross_entropy(output, label)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
            label.fill_(real_label)
            lossGA = F.binary_cross_entropy(output, label)
            lossG += lossGA.item()
            lossGA.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))    
        

## model changed +Dp

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netp = NetP(nc=1)
neta = NetA(nc=1)
netd = NetD(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netp.children())[0].children(),
    list(neta.children())[0].children(),
    list(netd.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'

netg = netg.to(device)
netp = netp.to(device)
neta = neta.to(device)
netd = netd.to(device)
netg.train()
netp.train()
neta.train()
netd.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimP = optim.Adam(netp.parameters(), lr=lr/3)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, target = {} \n'.format(
            epoches, lr, batchSize, target))

print('Training starts')
low_loss = 10
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, ass_img, img, noass_img) in enumerate(train_loader): 
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        
        # update D
        lossD = 0
        optimD.zero_grad()
        output = netd(ass_label)
        label.fill_(real_label)
        lossD_real1 = F.binary_cross_entropy(output, label)
        lossD += lossD_real1.item()
        lossD_real1.backward()

        label.fill_(real_label)
        output1 = netd(noass_label)
        lossD_real2 = F.binary_cross_entropy(output1, label)
        lossD += lossD_real2.item()
        lossD_real2.backward()

        fake = netg(img).detach()
        label.fill_(fake_label)
        output2 = netd(fake)
        lossD_fake = F.binary_cross_entropy(output2, label)
        lossD += lossD_fake.item()
        lossD_fake.backward()

        optimD.step()


        # update P
        lossP = 0
        optimP.zero_grad()
        fake = netg(img).detach()
        assd = th.cat((img, ass_img), 1)
        noassd = th.cat((img, noass_img), 1)
        faked = th.cat((img, fake), 1)

        label.fill_(real_label)
        output1 = neta(assd)
        lossP_real1 = F.binary_cross_entropy(output1, label)
        lossP += lossP_real1.item()/2
        lossP_real1 = lossP_real1/2
        lossP_real1.backward()

        label.fill_(fake_label)
        output = neta(noassd)
        lossP_real2 = F.binary_cross_entropy(output, label)
        lossP += lossP_real2.item()/2
        lossP_real2 = lossP_real2/2
        lossP_real2.backward()

        label.fill_(fake_label)
        output = neta(faked)
        lossP_fake = F.binary_cross_entropy(output, label)
        lossP += lossP_fake.item()/2
        lossP_fake = lossP_fake/2
        lossP_fake.backward()

        optimP.step()
    

        # update A
        lossA = 0
        optimA.zero_grad()
        fake = netg(img).detach()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        faked = th.cat((img, fake), 1)

        label.fill_(real_label)
        output1 = neta(assd)
        lossA_real1 = F.binary_cross_entropy(output1, label)
        lossA += lossA_real1.item()
        lossA_real1.backward()

        label.fill_(fake_label)
        output = neta(noassd)
        lossA_real2 = F.binary_cross_entropy(output, label)
        lossA += lossA_real2.item()
        lossA_real2.backward()

        label.fill_(fake_label)
        output = neta(faked)
        lossA_fake = F.binary_cross_entropy(output, label)
        lossA += lossA_fake.item()
        lossA_fake.backward()

        optimA.step()

        # update G
        lossG = 0
        optimG.zero_grad()
        
        fake = netg(img)
        output = netd(fake)
        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward(retain_graph=True)
        
        faked = th.cat((img, fake), 1)
        output = netp(faked)
        label.fill_(real_label)
        lossGP = F.binary_cross_entropy(output, label)
        lossG += lossGP.item()/2
        lossGP = lossGP/2
        lossGP.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch, epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3, lossP/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD', 'lossP']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch, epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3, lossP/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
            
    if (epoch>= 300) and epoch%20==0:  
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict(),
            'netP': netp.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
    
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict(),
            'netP': netp.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {}  \n'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
        



## model only D_A

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
g_k = 2
target = '090'

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
# netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
# optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, g_k = {}, target={} \n'.format(
            epoches, lr, batchSize, g_k, target))

low_loss = 10
print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#     for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
#         noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
#         ass_img = ass_img.to(device).to(th.float32)
        
        if i % g_k ==0:
#             # update D
#             lossD = 0
#             optimD.zero_grad()
#             output = netd(ass_label)
#             label.fill_(real_label)
#             lossD_real1 = F.binary_cross_entropy(output, label)
#             lossD += lossD_real1.item()
#             lossD_real1.backward()

#             label.fill_(real_label)
#             output1 = netd(noass_label)
#             lossD_real2 = F.binary_cross_entropy(output1, label)
#             lossD += lossD_real2.item()
#             lossD_real2.backward()

#             fake = netg(img).detach()
#             label.fill_(fake_label)
#             output2 = netd(fake)
#             lossD_fake = F.binary_cross_entropy(output2, label)
#             lossD += lossD_fake.item()
#             lossD_fake.backward()

#             optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
#         if i % k ==0: 
        # update G
        lossG = 0
        optimG.zero_grad()
        fake = netg(img)
        output = netd(fake)

        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        ]]),
                           Y=np.array([[lossG/2, lossA/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  ]]),
                     Y=np.array([[lossG/2, lossA/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {} \n'.format(
            epoch, lossG/2, lossA/3,
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}'.format(
            epoch, lossG/2, lossA/3, 
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}\n'.format(
            epoch, lossG/2, lossA/3
        ))    
        

# GaitGAN and triplet 

In [ ]:
%load_ext autoreload
%autoreload 2

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '036'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 5
n_g = 0
n_d = 0


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=glr/2)
optimD = optim.Adam(netd.parameters(), lr=dlr/3)
optimA = optim.Adam(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        if i % n_g==0:
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake, _ = netg(img)
            label.fill_(fake_label)
            output2 = netd(fake.detach())
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake,_ = netg(img)
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked.detach())
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
        if i % n_d ==0: 
            # update G
            lossG = 0
            optimG.zero_grad()
            fake, A = netg(img)
            output = netd(fake)

            label.fill_(real_label)
            lossGD = F.binary_cross_entropy(output, label)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
            label.fill_(real_label)
            lossGA = F.binary_cross_entropy(output, label)
            lossG += lossGA.item()
            lossGA.backward()

            # constrain on generator
            fake_ass, P = netg(ass_label)
            fake_noass, N = netg(noass_label)
            lossTriplet = F.triplet_margin_loss(fake, fake_ass, fake_noass, margin = margin)
            lossG += lossTriplet.item()
            lossTriplet.backward()

            # constrain on encoder
    #         __, P = netg(ass_label)
    #         __, N = netg(noass_label)
    #         lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin)
    #         lossG += lossTriplet.item()
    #         lossTriplet.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
    


# GaitGAN k times and triplet   

In [ ]:
%load_ext autoreload
%autoreload 2

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 10
n_g = 2
n_d = 2


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=glr/3)
optimD = optim.Adam(netd.parameters(), lr=dlr/3)
optimA = optim.Adam(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         com_img = th.cat((noass_img, img, ass_img), 0)
#         com_label = th.cat(( label_neg, label_anc, label_pos), 0)
#         com_img = com_img.to(device).to(th.float32)
#         com_label = com_label.to(device).to(th.float32)
        
#         if(i ==0):
#             print(label_neg,label_anc,label_pos)
#             print(com_label)
#         print("shape",ass_label.shape,noass_label.shape,img.shape,com_img.shape, com_label.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)
        
        # update D
        if i % n_g==0:
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake, _ = netg(img)
            label.fill_(fake_label)
            output2 = netd(fake.detach()) #需要 detach 因為不希望更新fake的參數
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake,_ = netg(img)
            faked = th.cat((img, fake.detach()), 1)  #需要 detach 因為不希望更新fake的參數

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
        if i % n_d ==0: 
            # update G
            lossG = 0
            optimG.zero_grad()
            fake, A = netg(img)
            output = netd(fake)

            label.fill_(real_label)
            lossGD = F.binary_cross_entropy(output, label)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)  ##這裡需要retain graph 因為他之後有需要fake，因此需要retain

            faked = th.cat((img, fake), 1)
            output = neta(faked)
            label.fill_(real_label)
            lossGA = F.binary_cross_entropy(output, label)
            lossG += lossGA.item()
            lossGA.backward()

            ## new tripletloss
            _, P = netg(ass_img)
            __, N = netg(noass_img)
            lossf = TripletLoss(margin) #因為之前的A都沒被使用過所以不會遺失，如果是拿fake ，LossGA就需要再retain graph
            lossTriplet =lossf(A, P, N)
            lossG += lossTriplet.item()
            lossTriplet.backward()

    #         ## new onlinetripletloss
    #         __, com = netg(com_img)
    #         loss_fn = OnlineTripletLoss(margin, SemihardNegativeTripletSelector(margin))
    #         lossTriplet,len_triplet = loss_fn(com, com_label)
    #         lossG += lossTriplet.item()
    #         lossTriplet.backward()
    # #         print(lossTriplet.item(),len_triplet)

    #         ## triplet loss
    #         __, P = netg(ass_img)
    #         __, N = netg(noass_img)
    #         lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin) #因為之前的A都沒被使用過所以不會遺失，如果是拿fake ，LossGA就需要再retain graph
    #         lossG += lossTriplet.item()
    #         lossTriplet.backward()
    # #         if i%10==0:
    # #             print("tripletloss ",lossTriplet.item())

            ## tripletloss no negative
    #         N_plus = th.zeros((A.size()), requires_grad=False).to(device)
    #         lossTriplet_AP = F.triplet_margin_loss(A, P, N_plus, margin = margin)
    #         lossG += lossTriplet_AP.item()
    #         lossTriplet += lossTriplet_AP
    #         lossTriplet.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG  Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
    


# from PixelDT code刪

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir)

iteration = 0
lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 128

real_label = th.zeros((batchSize, 1), requires_grad=False).to(device)
fake_label = th.ones((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr)
optimD = optim.Adam(netd.parameters(), lr=lr)
optimA = optim.Adam(neta.parameters(), lr=lr)

print('Training starts')
while iteration < 40000:
    ass_label, noass_label, img = dataset.getbatch(batchSize)
    ass_label = ass_label.to(device).to(th.float32)
    noass_label = noass_label.to(device).to(th.float32)
    img = img.to(device).to(th.float32)
    
    # update D
    lossD = 0
    optimD.zero_grad()
    output = netd(ass_label)
#     label.fill_(real_label)
    lossD_real1 = F.binary_cross_entropy(output, real_label)
#     lossD += lossD_real1.item()
#     lossD_real1.backward()

#     label.fill_(real_label)
    output1 = netd(noass_label)
    lossD_real2 = F.binary_cross_entropy(output1, real_label)
#     lossD += lossD_real2.item()
#     lossD_real2.backward()

    fake = netg(img).detach()
#     label.fill_(fake_label)
    output2 = netd(fake)
    lossD_fake = F.binary_cross_entropy(output2, fake_label)
#     lossD += lossD_fake.item()
#     lossD_fake.backward()
    lossD = (lossD_real1+ lossD_real2+ lossD_fake)/3
    lossD.backward()

    lossD_item = lossD.item()
    optimD.step()
    
    # update A
    lossA = 0
    optimA.zero_grad()
    assd = th.cat((img, ass_label), 1)
    noassd = th.cat((img, noass_label), 1)
    fake = netg(img).detach()
    faked = th.cat((img, fake), 1)

#     label.fill_(real_label)
    output = neta(assd)
    lossA_real1 = F.binary_cross_entropy(output, real_label)
#     lossA += lossA_real1.item()
#     lossA_real1.backward()

#     label.fill_(fake_label)
    output1 = neta(noassd)
    lossA_real2 = F.binary_cross_entropy(output1, fake_label)
#     lossA += lossA_real2.item()
#     lossA_real2.backward()

#     label.fill_(fake_label)
    output2 = neta(faked)
    lossA_fake = F.binary_cross_entropy(output2, fake_label)
#     lossA += lossA_fake.item()
#     lossA_fake.backward()
    lossA = (lossA_real1+ lossA_real2 +lossA_fake)/3
    lossA.backward()
    
    lossA_item = lossA.item()
    optimA.step()
    
    # update G
    lossG = 0
    optimG.zero_grad()
    fake = netg(img)
    output = netd(fake)
#     label.fill_(real_label)
    lossGD = F.binary_cross_entropy(output, real_label)
#     lossG += lossGD.item()
#     lossGD.backward(retain_graph=True)

    faked = th.cat((img, fake), 1)
    output = neta(faked)
#     label.fill_(real_label)
    lossGA = F.binary_cross_entropy(output, real_label)
#     lossG += lossGA.item()
#     lossGA.backward()
    lossG = (lossGD + lossGA)/2
    lossG.backward()
    
    lossG_item = lossG.item()
    optimG.step()

    iteration += 1
    
    if iteration % 20 == 0:
        with th.no_grad():
            netg.eval()  #切換
            fake = netg(img) 
            netg.train() #切換回去
        fake = (fake + 1) / 2 * 255
        real = (ass_label + 1) / 2 * 255
        ori = (img + 1) / 2 * 255
        al = th.cat((fake, real, ori), 2)
        display = make_grid(al, 20).cpu().numpy()
        if win1 is None:
            win1 = vis.image(display,
                             opts=dict(title="train", caption='train'))
        else:
            vis.image(display, win=win1)
    if iteration % 5000 == 0 or iteration==500:
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_Name+'/snapshot'+ Model_Name +'_%d.t7' % iteration)
        with open(Model_Name+"/snapshot_log.txt", "a") as myfile:
            myfile.write('iter = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            iteration, lossG_item, lossA_item, lossD_item
        ))
        print('iter = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            iteration, lossG_item, lossA_item, lossD_item
        ))
    if iteration % 5000==0 or iteration==10 or iteration==500:
        if win is None:
            win = vis.line(X=np.array([[iteration, iteration,
                                        iteration]]),
                           Y=np.array([[lossG_item, lossA_item, lossD_item]]),
                           opts=dict(
                               title='GaitGAN',
                               ylabel='loss',
                               xlabel='iterations',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[iteration, iteration,
                                  iteration]]),
                     Y=np.array([[lossG_item, lossA_item, lossD_item]]),
                     win=win,
                     update='append')


# WGAN 

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 0
n_g = 1
n_d = 2
clip = 0.1


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.RMSprop(netg.parameters(), lr=glr/2)
optimD = optim.RMSprop(netd.parameters(), lr=dlr/3)
optimA = optim.RMSprop(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={},clip={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp, clip))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        

        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        if i % n_g==0:
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
    #         label.fill_(real_label)
    #         lossD_real1 = F.binary_cross_entropy(output, label)
            lossD_real1 = -th.mean(output)
            lossD += lossD_real1.item()
            lossD_real1.backward()

    #         label.fill_(real_label)
            output1 = netd(noass_label)
    #         lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD_real2 = -th.mean(output1)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake = netg(img).detach()
    #         label.fill_(fake_label)
            output2 = netd(fake)
    #         lossD_fake = F.binary_cross_entropy(output2, label)
            lossD_fake = th.mean(output2)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            for p in netd.parameters():
                p.data.clamp_(-clip, clip)

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

    #         label.fill_(real_label)
            output1 = neta(assd)
    #         lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA_real1 = -th.mean(output1)
            lossA += lossA_real1.item()
            lossA_real1.backward()

    #         label.fill_(fake_label)
            output = neta(noassd)
    #         lossA_real2 = F.binary_cross_entropy(output, label)
            lossA_real2 = th.mean(output)
            lossA += lossA_real2.item()
            lossA_real2.backward()

    #         label.fill_(fake_label)
            output = neta(faked)
    #         lossA_fake = F.binary_cross_entropy(output, label)
            lossA_fake = th.mean(output)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()

            for p in neta.parameters():
                p.data.clamp_(-clip, clip)

        # update G
        if i % n_d == 0:
            lossG = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

#             label.fill_(real_label)
#             lossGD = F.binary_cross_entropy(output, label)
            lossGD = -th.mean(output)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
#             label.fill_(real_label)
#             lossGA = F.binary_cross_entropy(output, label)
            lossGA = -th.mean(output)
            lossG += lossGA.item()
            lossGA.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
        
        


n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts
Epoch = 2, ErrG = 3.752237558364868, ErrA = -1.8985527356465657, ErrD = -3.5465147892634072
Epoch = 4, ErrG = 5.705430269241333, ErrA = -2.807316621144613, ErrD = -5.611066023508708
Epoch = 6, ErrG = 4.896071195602417, ErrA = -3.07237180074056, ErrD = -6.601948658625285
Epoch = 8, ErrG = 8.87260103225708, ErrA = -4.403616587320964, ErrD = -9.87374464670817
Epoch = 10, ErrG = 7.947443723678589, ErrA = -4.1470723152160645, ErrD = -8.064921021461487
Epoch = 12, ErrG = 1.238088607788086, ErrA = -4.921435038248698, ErrD = -4.710442225138347
Epoch = 14, ErrG = 0.5194168090820312, ErrA = -5.6255998611450195, ErrD = -5.323735237121582
Epoch = 16, ErrG = 6.348399639129639, ErrA = -7.5277970631917315, ErrD = -7.016017913818359
Epoch = 18, ErrG = 0.8842306137084961, ErrA = -7.25491460164388, ErrD = -7.062686284383138
Epoch = 20, ErrG = 0.0035724639892578125, ErrA = -7.803771336873372, ErrD = -7.927731831868489
Epoch = 22, E

 # WGAN-GP

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 10
beta1 = 0
beta2 = 0.999
margin = 0
n_g = 0
n_d = 5

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        
        # update D
        if i % n_g==0:
            lossD = 0
            lossD_ = 0
            optimD.zero_grad()
            output = netd(ass_label)
    #         label.fill_(real_label)
    #         lossD_real1 = F.binary_cross_entropy(output, label)
            lossD_real1 = -th.mean(output)
            lossD_ += lossD_real1
            lossD += lossD_real1.item()
    #         lossD_real1.backward()

    #         label.fill_(real_label)
            output1 = netd(noass_label)
    #         lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD_real2 = -th.mean(output1)
            lossD_ += lossD_real2
            lossD += lossD_real2.item()
    #         lossD_real2.backward()

            fake = netg(img).detach()
    #         label.fill_(fake_label)
            output2 = netd(fake)
    #         lossD_fake = F.binary_cross_entropy(output2, label)
            lossD_fake = th.mean(output2)
            lossD_ += lossD_fake
            lossD += lossD_fake.item()
            gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
    #         lossD_fake.backward()
            lossD_ = lossD_/3 + lambda_gp * gradient_penalty
            lossD_.backward()

            optimD.step()


            # update A
            lossA = 0
            lossA_ = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

    #         label.fill_(real_label)
            output1 = neta(assd)
    #         lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA_real1 = -th.mean(output1)
            lossA += lossA_real1.item()
            lossA_ += lossA_real1
    #         lossA_real1.backward()

    #         label.fill_(fake_label)
            output = neta(noassd)
    #         lossA_real2 = F.binary_cross_entropy(output, label)
            lossA_real2 = th.mean(output)
            lossA += lossA_real2.item()
            lossA_ += lossA_real2
    #         lossA_real2.backward()

    #         label.fill_(fake_label)
            output = neta(faked)
    #         lossA_fake = F.binary_cross_entropy(output, label)
            lossA_fake = th.mean(output)
            lossA += lossA_fake.item()
            lossA_ += lossA_fake
    #         lossA_fake.backward()
            gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
            lossA_ = lossA_/3 + lambda_gp * gradient_penalty

            lossA_.backward()
            optimA.step()
    
            
        # update G
        if i % n_d == 0:
            lossG = 0
            lossG_ = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

#             label.fill_(real_label)
#             lossGD = F.binary_cross_entropy(output, label)
            lossGD = -th.mean(output)
            lossG += lossGD.item()
            lossG_ += lossGD
#             lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
#             label.fill_(real_label)
#             lossGA = F.binary_cross_entropy(output, label)
            lossGA = -th.mean(output)
            lossG += lossGA.item()
            lossG_ += lossGA
#             lossGA.backward()
            lossG_ = lossG_/2
            lossG_.backward(retain_graph=True)
            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))           

        
        


# SA GaitGAN (hing)

In [2]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        

epoches = 700
glr = 0.00001
dlr = 0.00004
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0.9
margin = 0
n_g = 2
n_d = 1


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        if i % n_g ==0:
            # update D
            lossD = 0
            lossD_ = 0
            optimD.zero_grad()
            d_out_assreal,dr1 = netd(ass_label)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()

            lossD_ += d_loss_assreal
            lossD += d_loss_assreal.item()

            d_out_noassreal,dr2 = netd(noass_label)
            d_loss_noassreal = nn.ReLU()(1.0 - d_out_noassreal).mean()

            lossD_ += d_loss_noassreal
            lossD += d_loss_noassreal.item()

            fake, gf1 = netg(img)
            d_out_fake, df1 = netd(fake.detach())  #需要 detach 因為不希望更新fake的參數
            d_loss_fake = nn.ReLU()(1.0 + d_out_fake).mean()

            lossD_ += d_loss_fake
            lossD += d_loss_fake.item()
    #         gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
            lossD_ = lossD_/3
            lossD_.backward()
            optimD.step()


            # update A
            lossA = 0
            lossA_ = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            faked, gf1 = netg(img)
            faked = th.cat((img, faked.detach()), 1)  #需要 detach 因為不希望更新fake的參數

            d_out_assreal,dr1 = neta(assd)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()
            lossA += d_loss_assreal.item()
            lossA_ += d_loss_assreal

            d_out_noassreal,dr2 = neta(noassd)
            d_loss_noassreal = nn.ReLU()(1.0 + d_out_noassreal).mean()

            lossA_ += d_loss_noassreal
            lossA += d_loss_noassreal.item()

            d_out_faked, df3 = neta(faked)
            d_loss_faked = nn.ReLU()(1.0 + d_out_faked).mean()

            lossA_ += d_loss_faked
            lossA += d_loss_faked.item()
    #         gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
            lossA_ = lossA_/3
            lossA_.backward()
            optimA.step()

            
        # update G
        if i % n_d == 0:
            lossG = 0
            lossG_ = 0
            optimG.zero_grad()
            fake,_= netg(img)
            g_out_fake,_ = netd(fake)
            g_loss_fake = - g_out_fake.mean()

            lossG += g_loss_fake.item()
            lossG_ += g_loss_fake

            faked = th.cat((img, fake), 1)
            g_out_faked,_ = neta(faked)
            g_loss_faked = - g_out_faked.mean()
            lossG += g_loss_faked.item()
            lossG_ += g_loss_faked

            lossG_ = lossG_/2
            lossG_.backward(retain_graph=True) ##其實這裡也不需要retain graph 因為他只 backward 一次；
            #如果爾後有要再backward，第一次就需要retain graph
            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/2, lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))           

        

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts
Epoch = 2, ErrG = 1.8996484279632568, ErrA = 0.617903919890523, ErrD = 0.02228272706270218, Gattn=-0.0034067004453390837, Dattn=0.001238542958162725, Aattn=-0.00010161368118133396
Epoch = 4, ErrG = 0.7542737275362015, ErrA = 0.6713284552097321, ErrD = 0.45941516508658725, Gattn=-0.005732168909162283, Dattn=0.001418719650246203, Aattn=-3.125678631477058e-05
Epoch = 6, ErrG = -0.08506286144256592, ErrA = 0.6819584127515554, ErrD = 0.7010320872068405, Gattn=-0.0071759894490242004, Dattn=0.0016002730699256063, Aattn=2.4452589059364982e-05
Epoch = 8, ErrG = -0.05347627401351929, ErrA = 0.6719489550838867, ErrD = 0.7059540351231893, Gattn=-0.00781609583646059, Dattn=0.0016207138542085886, Aattn=0.0002464703575242311
Epoch = 10, ErrG = -0.026103854179382324, ErrA = 0.6940227796634039, ErrD = 0.7041413262486458, Gattn=-0.0080963633954

Epoch = 94, ErrG = 0.6470556482672691, ErrA = 0.5078609151144823, ErrD = 0.4706652561823527, Gattn=-0.011430235579609871, Dattn=0.018935296684503555, Aattn=0.04463759809732437
Epoch = 96, ErrG = 0.004471957683563232, ErrA = 0.3324270291874806, ErrD = 0.3612421900033951, Gattn=-0.011442427523434162, Dattn=0.019414205104112625, Aattn=0.046229250729084015
Epoch = 98, ErrG = 0.9115673899650574, ErrA = 0.5088222300012907, ErrD = 0.4312947802245617, Gattn=-0.011487491428852081, Dattn=0.0196624007076025, Aattn=0.047660376876592636
Epoch = 100, ErrG = 1.221674919128418, ErrA = 0.44035592178503674, ErrD = 0.6016119252890348, Gattn=-0.011491957120597363, Dattn=0.020076587796211243, Aattn=0.0494144931435585
Epoch = 102, ErrG = 0.687450647354126, ErrA = 0.487853745619456, ErrD = 0.4375476986169815, Gattn=-0.011560247279703617, Dattn=0.020314455032348633, Aattn=0.05109594017267227
Epoch = 104, ErrG = 0.9769279956817627, ErrA = 0.5190529078245163, ErrD = 0.3872065593798955, Gattn=-0.0116104232147336

Epoch = 188, ErrG = 0.6759753674268723, ErrA = 0.3857127403219541, ErrD = 0.3561445126930873, Gattn=-0.013287266716361046, Dattn=0.02251395583152771, Aattn=0.09869275987148285
Epoch = 190, ErrG = -0.34217745065689087, ErrA = 0.48853286045293015, ErrD = 0.5048794994751612, Gattn=-0.013311813585460186, Dattn=0.02252340316772461, Aattn=0.0994398221373558
Epoch = 192, ErrG = 1.00471830368042, ErrA = 0.44253737355271977, ErrD = 0.27458227674166363, Gattn=-0.01331554725766182, Dattn=0.02247766964137554, Aattn=0.10004102438688278
Epoch = 194, ErrG = 0.7553345859050751, ErrA = 0.47755634722610313, ErrD = 0.4919276957710584, Gattn=-0.013378213159739971, Dattn=0.022476568818092346, Aattn=0.10103055089712143
Epoch = 196, ErrG = 0.71193066239357, ErrA = 0.545421801507473, ErrD = 0.45395897328853607, Gattn=-0.013416052795946598, Dattn=0.022486018016934395, Aattn=0.10193054378032684
Epoch = 198, ErrG = 1.7210158705711365, ErrA = 0.42164099713166553, ErrD = 0.2774216930071513, Gattn=-0.01344761624932

Epoch = 282, ErrG = 1.5556307435035706, ErrA = 0.4463205710053444, ErrD = 0.31928500284751254, Gattn=-0.015047806315124035, Dattn=0.020505735650658607, Aattn=0.12681598961353302
Epoch = 284, ErrG = 1.2538426518440247, ErrA = 0.2810995342830817, ErrD = 0.31066010457774, Gattn=-0.01509208232164383, Dattn=0.020398883149027824, Aattn=0.12747325003147125
Epoch = 286, ErrG = 0.967411682009697, ErrA = 0.2551736906170845, ErrD = 0.24104593694210052, Gattn=-0.01512041687965393, Dattn=0.020315047353506088, Aattn=0.1280733197927475
Epoch = 288, ErrG = 1.8535668849945068, ErrA = 0.4252461517850558, ErrD = 0.38152264927824336, Gattn=-0.015167401172220707, Dattn=0.020258860662579536, Aattn=0.12857785820960999
Epoch = 290, ErrG = 0.6663249433040619, ErrA = 0.27613263328870136, ErrD = 0.23087313771247864, Gattn=-0.015194619074463844, Dattn=0.020278174430131912, Aattn=0.1289464831352234
Epoch = 292, ErrG = 1.5400712490081787, ErrA = 0.33082864930232364, ErrD = 0.32124556849400204, Gattn=-0.015235682949

Epoch = 376, ErrG = 2.3277333974838257, ErrA = 0.28991857171058655, ErrD = 0.09432371705770493, Gattn=-0.01715625450015068, Dattn=0.01855456456542015, Aattn=0.14471375942230225
Epoch = 378, ErrG = 0.8720053434371948, ErrA = 0.1527638422946135, ErrD = 0.3949868530035019, Gattn=-0.017211779952049255, Dattn=0.018550043925642967, Aattn=0.1450711488723755
Epoch = 380, ErrG = 2.8400436639785767, ErrA = 0.3514976470420758, ErrD = 0.17523590723673502, Gattn=-0.017252475023269653, Dattn=0.018460793420672417, Aattn=0.14536993205547333
Epoch = 382, ErrG = 2.4275747537612915, ErrA = 0.1701316460967064, ErrD = 0.04692357654372851, Gattn=-0.017306765541434288, Dattn=0.01838310807943344, Aattn=0.14577271044254303
Epoch = 384, ErrG = 2.262136220932007, ErrA = 0.36663631722331047, ErrD = 0.1500918666521708, Gattn=-0.017352014780044556, Dattn=0.018454043194651604, Aattn=0.14621219038963318
Epoch = 386, ErrG = 1.5769317150115967, ErrA = 0.21206103513638178, ErrD = 0.08238822221755981, Gattn=-0.0174101591

Epoch = 470, ErrG = 2.577104091644287, ErrA = 0.1917615681886673, ErrD = 0.02472281518081824, Gattn=-0.019432472065091133, Dattn=0.016977449879050255, Aattn=0.1597951501607895
Epoch = 472, ErrG = 1.9232202768325806, ErrA = 0.31466099930306274, ErrD = 0.1260106364885966, Gattn=-0.019514458253979683, Dattn=0.016977353021502495, Aattn=0.16015253961086273
Epoch = 474, ErrG = 2.4668508768081665, ErrA = 0.12074443697929382, ErrD = 0.11391653617223103, Gattn=-0.01956813782453537, Dattn=0.016921386122703552, Aattn=0.16036516427993774
Epoch = 476, ErrG = 2.2352635860443115, ErrA = 0.06397978713115056, ErrD = 0.04118705044190089, Gattn=-0.01961853727698326, Dattn=0.01690421998500824, Aattn=0.16071023046970367
Epoch = 478, ErrG = 1.7247258126735687, ErrA = 0.20517059167226157, ErrD = 0.12780851125717163, Gattn=-0.019655214622616768, Dattn=0.016919000074267387, Aattn=0.1611608862876892
Epoch = 480, ErrG = 1.457535445690155, ErrA = 0.18085120618343353, ErrD = 0.18368761241436005, Gattn=-0.019703712

Epoch = 564, ErrG = 2.9840166568756104, ErrA = 0.23537293883661428, ErrD = 0.030452149609724682, Gattn=-0.02156555838882923, Dattn=0.01581835374236107, Aattn=0.1732722818851471
Epoch = 566, ErrG = 2.473421812057495, ErrA = 0.1275919501980146, ErrD = 0.04912359640002251, Gattn=-0.02162042260169983, Dattn=0.01576497219502926, Aattn=0.1735120564699173
Epoch = 568, ErrG = 3.299262523651123, ErrA = 0.20995665589968363, ErrD = 0.039668520291646324, Gattn=-0.021638594567775726, Dattn=0.015688465908169746, Aattn=0.17374356091022491
Epoch = 570, ErrG = 2.7525181770324707, ErrA = 0.17016186813513437, ErrD = 0.03715775596598784, Gattn=-0.021681267768144608, Dattn=0.015672758221626282, Aattn=0.17396995425224304
Epoch = 572, ErrG = 2.8531715869903564, ErrA = 0.25313299149274826, ErrD = 0.05841648702820142, Gattn=-0.021715225651860237, Dattn=0.015561922453343868, Aattn=0.17432363331317902
Epoch = 574, ErrG = 2.450057864189148, ErrA = 0.07816040515899658, ErrD = 0.07907374699910481, Gattn=-0.02179518

Epoch = 658, ErrG = 3.0978980660438538, ErrA = 0.1924865779777368, ErrD = 0.06667035321394603, Gattn=-0.023618437349796295, Dattn=0.01440544705837965, Aattn=0.18596096336841583
Epoch = 660, ErrG = 2.274911880493164, ErrA = 0.05331048804024855, ErrD = 0.06769030727446079, Gattn=-0.023651763796806335, Dattn=0.014346727170050144, Aattn=0.18619690835475922
Epoch = 662, ErrG = 2.2050634622573853, ErrA = 0.1165768491725127, ErrD = 0.08983064691225688, Gattn=-0.023709945380687714, Dattn=0.014274507761001587, Aattn=0.18638008832931519
Epoch = 664, ErrG = 2.3505425453186035, ErrA = 0.15459846208492914, ErrD = 0.013870025674502054, Gattn=-0.0237516388297081, Dattn=0.014064074493944645, Aattn=0.1867741197347641
Epoch = 666, ErrG = 2.161745846271515, ErrA = 0.20249265929063162, ErrD = 0.04775057236353556, Gattn=-0.02379130944609642, Dattn=0.01412009634077549, Aattn=0.18691669404506683
Epoch = 668, ErrG = 3.1226245164871216, ErrA = 0.1581517687688271, ErrD = 0.06907273332277934, Gattn=-0.0238175895

# SA GaitGAN (WGAN-GP) 刪

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00001
dlr = 0.00004
# lr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
n_critic = 0
target = '090'
lambda_gp = 10
beta1 = 0
beta2 = 0.9

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_critic = {}, target={},lambda_gp={} \n'.format(
            epoches, dlr, glr, batchSize, beta1, beta2, n_critic, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        
        
        # update D
        lossD = 0
        lossD_ = 0
        optimD.zero_grad()
        d_out_assreal,dr1 = netd(ass_label)
        d_loss_assreal = -th.mean(d_out_assreal)

        lossD_ += d_loss_assreal
        lossD += d_loss_assreal.item()

        d_out_noassreal,dr2 = netd(noass_label)
        d_loss_noassreal = -th.mean(d_out_noassreal)

        lossD_ += d_loss_noassreal
        lossD += d_loss_noassreal.item()

        fake, gf1 = netg(img)
        d_out_fake, df1 = netd(fake.detach())  #需要 detach 因為不希望更新fake的參數
        d_loss_fake = th.mean(d_out_fake)

        lossD_ += d_loss_fake
        lossD += d_loss_fake.item()
        gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
        lossD_ = lossD_/3 + lambda_gp * gradient_penalty
#         lossD_ = lossD_/3
        lossD_.backward()
        optimD.step()
        
            
        # update A
        lossA = 0
        lossA_ = 0
        optimA.zero_grad()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        faked, gf1 = netg(img)
        faked = th.cat((img, faked.detach()), 1)  #需要 detach 因為不希望更新fake的參數

        d_out_assreal,dr1 = neta(assd)
        d_loss_assreal = -th.mean(d_out_assreal)
        lossA += d_loss_assreal.item()
        lossA_ += d_loss_assreal

        d_out_noassreal,dr2 = neta(noassd)
        d_loss_noassreal = th.mean(d_out_noassreal)

        lossA_ += d_loss_noassreal
        lossA += d_loss_noassreal.item()
        
        d_out_faked, df3 = neta(faked)
        d_loss_faked = th.mean(d_out_faked)

        lossA_ += d_loss_faked
        lossA += d_loss_faked.item()
        gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
        lossA_ = lossA_/3 + lambda_gp * gradient_penalty
#         lossA_ = lossA_/3
        lossA_.backward()
        optimA.step()

            
        # update G
#         if i % n_critic == 0:
        lossG = 0
        lossG_ = 0
        optimG.zero_grad()
        fake,_= netg(img)
        g_out_fake,_ = netd(fake)
        g_loss_fake = - g_out_fake.mean()

        lossG += g_loss_fake.item()
        lossG_ += g_loss_fake

        faked = th.cat((img, fake), 1)
        g_out_faked,_ = neta(faked)
        g_loss_faked = - g_out_faked.mean()
        lossG += g_loss_faked.item()
        lossG_ += g_loss_faked
        
        lossG_ = lossG_/2
        lossG_.backward(retain_graph=True) ##其實這裡也不需要retain graph 因為他只 backward 一次；
        #如果爾後有要再backward，第一次就需要retain graph
        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/2, lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))           

        

# SA GaitGAN and triplet 

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

epoches = 700
glr = 0.00001
dlr = 0.00004
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0.9
margin = 10
n_g = 2
n_d = 0

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)

        if i % n_g ==0:
            # update D
            lossD = 0
            lossD_ = 0
            optimD.zero_grad()
            d_out_assreal,dr1 = netd(ass_label)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()

            lossD_ += d_loss_assreal
            lossD += d_loss_assreal.item()

            d_out_noassreal,dr2 = netd(noass_label)
            d_loss_noassreal = nn.ReLU()(1.0 - d_out_noassreal).mean()

            lossD_ += d_loss_noassreal
            lossD += d_loss_noassreal.item()

            fake, code = netg(img)
            d_out_fake, df1 = netd(fake.detach())
            d_loss_fake = nn.ReLU()(1.0 + d_out_fake).mean()

            lossD_ += d_loss_fake
            lossD += d_loss_fake.item()
    #         gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
            lossD_ = lossD_/3
            lossD_.backward()
            optimD.step()


            # update A
            lossA = 0
            lossA_ = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            faked, code = netg(img)
            faked = th.cat((img, faked.detach()), 1)

            d_out_assreal,dr1 = neta(assd)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()
            lossA += d_loss_assreal.item()
            lossA_ += d_loss_assreal

            d_out_noassreal,dr2 = neta(noassd)
            d_loss_noassreal = nn.ReLU()(1.0 + d_out_noassreal).mean()

            lossA_ += d_loss_noassreal
            lossA += d_loss_noassreal.item()

            d_out_faked, df3 = neta(faked)
            d_loss_faked = nn.ReLU()(1.0 + d_out_faked).mean()

            lossA_ += d_loss_faked
            lossA += d_loss_faked.item()
    #         gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
            lossA_ = lossA_/3
            lossA_.backward()
            optimA.step()

            
        # update G
        if i % n_d == 0:
            lossG = 0
            lossG_ = 0
            optimG.zero_grad()
            fake, A= netg(img)
            g_out_fake,_ = netd(fake)
            g_loss_fake = - g_out_fake.mean()

            lossG += g_loss_fake.item()
            lossG_ += g_loss_fake

            faked = th.cat((img, fake), 1)
            g_out_faked,_ = neta(faked)
            g_loss_faked = - g_out_faked.mean()
            lossG += g_loss_faked.item()
            lossG_ += g_loss_faked

            # constrain on generator
            fake_ass, P = netg(ass_img)
            fake_noass, N = netg(noass_img)
            lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin)
            lossG_ += lossTriplet
            lossG += lossTriplet.item()
    #         lossTriplet.backward()

            lossG_ = lossG_/3
            lossG_.backward(retain_graph=True)
            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrTri = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/3,lossTriplet.item(), lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/3)<low_loss:  
        low_loss = lossG/3
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))           

# SA GaitGAN and triplet (onlineloss)

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00001
dlr = 0.00004
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0.9
margin = 10
n_g = 2
n_d = 0

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, noass_img, img, ass_img, label_neg, label_anc, label_pos) in enumerate(train_loader):
        
        com_img = th.cat((noass_img, img, ass_img), 0)
        com_label = th.cat(( label_neg, label_anc, label_pos), 0)
        com_img = com_img.to(device).to(th.float32)
        com_label = com_label.to(device).to(th.float32)
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        

        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)

        if i % n_g ==0:
            # update D
            lossD = 0
            lossD_ = 0
            optimD.zero_grad()
            d_out_assreal,dr1 = netd(ass_label)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()

            lossD_ += d_loss_assreal
            lossD += d_loss_assreal.item()

            d_out_noassreal,dr2 = netd(noass_label)
            d_loss_noassreal = nn.ReLU()(1.0 - d_out_noassreal).mean()

            lossD_ += d_loss_noassreal
            lossD += d_loss_noassreal.item()

            fake, code = netg(img)
            d_out_fake, df1 = netd(fake.detach())
            d_loss_fake = nn.ReLU()(1.0 + d_out_fake).mean()

            lossD_ += d_loss_fake
            lossD += d_loss_fake.item()
    #         gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
            lossD_ = lossD_/3
            lossD_.backward()
            optimD.step()


            # update A
            lossA = 0
            lossA_ = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            faked, code = netg(img)
            faked = th.cat((img, faked.detach()), 1)

            d_out_assreal,dr1 = neta(assd)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()
            lossA += d_loss_assreal.item()
            lossA_ += d_loss_assreal

            d_out_noassreal,dr2 = neta(noassd)
            d_loss_noassreal = nn.ReLU()(1.0 + d_out_noassreal).mean()

            lossA_ += d_loss_noassreal
            lossA += d_loss_noassreal.item()

            d_out_faked, df3 = neta(faked)
            d_loss_faked = nn.ReLU()(1.0 + d_out_faked).mean()

            lossA_ += d_loss_faked
            lossA += d_loss_faked.item()
    #         gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
            lossA_ = lossA_/3
            lossA_.backward()
            optimA.step()


            
        # update G
        if i % n_d == 0:
            lossG = 0
            lossG_ = 0
            optimG.zero_grad()
            fake, A= netg(img)
            g_out_fake,_ = netd(fake)
            g_loss_fake = - g_out_fake.mean()

            lossG += g_loss_fake.item()
            lossG_ += g_loss_fake

            faked = th.cat((img, fake), 1)
            g_out_faked,_ = neta(faked)
            g_loss_faked = - g_out_faked.mean()
            lossG += g_loss_faked.item()
            lossG_ += g_loss_faked

    #         # constrain on encoder
    #         fake_ass, P = netg(ass_img)
    #         fake_noass, N = netg(noass_img)
    #         lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin)
    #         lossG_ += lossTriplet
    #         lossG += lossTriplet.item()
    # #         lossTriplet.backward()

            ## new onlinetripletloss
            __, com = netg(com_img)
            loss_fn = OnlineTripletLoss(margin, SemihardNegativeTripletSelector(margin))
            lossTriplet,len_triplet = loss_fn(com, com_label)
            lossG += lossTriplet.item()
            lossG_ += lossTriplet
    #         lossTriplet.backward()


            lossG_ = lossG_/3
            lossG_.backward(retain_graph=True)
            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrTri = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/3,lossTriplet.item(), lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/3)<low_loss:  
        low_loss = lossG/3
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))           